# **Example 4** - Measures and Strategies

In this notebook we will show you all the different options and inputs for creating different measures and how to create a strategy from the different measures. 

**Measures** in FloodAdapt can be categorized into **two** different categories, which we will cover all in this notebook:

**1. Hazard** - Mitigate the hazard directly

- Floodwall
- Pump
- Green Infrastructure

**2. Impact**  - Mitiage the impacts of the hazard on the assets

- Elevate properties 
- Buyout Properties
- Floodproof Properties

**Strategies** consist of multiple measures combined for example: ***StrategyXYZ* = *Floodwall + Elevated Homes + Pumps***

If you want to learn in more detail about measures and what a strategy is, please read the sections on [**Measures**](https://deltares-research.github.io/FloodAdapt/4_user_guide/measures/) and [**Strategies**](https://deltares-research.github.io/FloodAdapt/4_user_guide/strategy/)  in the FloodAdapt GUI documentation.

## **Step 0**: Import packages

In [1]:
import pandas as pd
from pathlib import Path

from flood_adapt.flood_adapt import FloodAdapt
from flood_adapt.objects.measures.measures import *
from flood_adapt import Settings
from flood_adapt.objects.strategies.strategies import Strategy
from flood_adapt import unit_system as us

## **Step 1**: Getting started - Initiate the database

Let's start with initiating the database and FloodAdapt class. 
1. Initiate the database class `Settings` by defining the `DATABASE_ROOT` and `DATABASE_NAME`.
2. Initiate the class `FloodAdapt` by parsing the `Settings().database_path`.

In [3]:
# Set up the settings for the database
Settings(
    DATABASE_ROOT=Path("../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

# Create the FloodAdapt instance
fa = FloodAdapt(Settings().database_path)

## **Step 2:** Create Hazard Measures
We can divide the hazard measures into: 
1. **Hydraulic** (grey) measures -  structures such as levees, pumps, seawalls that protect infrastructure and residents
2. **Green infrastructure** measures - integrate natural elements like parks, green roofs, and permeable pavements into city

To create a hazard measure in Floodadapt we need to create a `Measure` object with the FloodAdapt classes. Each object has the following parameters:

**Basic inputs**   
Basic inputs for <u>all</u> types of hazard measures are:
- `name`: The name of the measure
- `description` (optionl): A description of the measure
- `type`: The type of measure
- `selection type`: The type of spatial shape of the measure*
- `polygon file`: The file path to the spatial file of the measure

****selection_type***  
The selection type describes in what type of spatial shape the measure is parsed. There are four options:
- `polyline`: This could be a floodwall or levee 
- `polygon`: This could be a reservoir, or a selection of assets within that polygon
- `aggregation_area`: Apply a measure to a whole neighourhood e.g. - elevate buildings
- `ALL`: Apply a measure to all the assets e.g. floodproof all buildings. 

**Measure specific inputs**  
For each hazard measure there are additional inputs that only apply to that measure in particular.   

- **Floodwall** specific inputs are:
    - `elevation`: The elevation value and unit (m,ft) of the floodwall
- **Pump** specific inputs are:
    - `discharge`: The discharge value and unit (cfs, cms)
- **Green Infrastructure** specific inputs are:
    - `volume`: The storage volume value and unit (cf, m3)
    - `height` (optional): The height of the greening or water square. In the greening measure, the height is used to validate the calculated total storage volume. 
    - `aggregation_area_type` (optional): The name of the aggregation area category.
    - `aggregation_area_name` (optional): The name of the specific aggregation area within the category.
    - `percent_area` (optional): The percentage of how much of a given area (polygon, aggregaton area) will be greened.

    The green infrastructure class can be divided into **sub-categories**:
    - `greening`: Greening a certain area e.g. aggregation area and define the percentage of how much of that area will be greened.
    - `total_storage`: Define a total storage volume.
    - `water_square`: Define the depth of the water square and the total storage volume will be calculated based on the polygon area and depth

Below we will create an `Measure`-object for each **hazard measure**.

In [4]:
# Create a measure object for a Floodwall
attrs_floodwall = FloodWall(
        name= "Seawall_12ft",
        description = "12ft Seawall",
        type = MeasureType.floodwall,
        selection_type = SelectionType.polyline,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\seawall\seawall.geojson"),
        elevation = us.UnitfulLength(value=12, units=us.UnitTypesLength.feet))

# Create a measure object for a Pump
attrs_pump = Pump(
        name= "Pump",
        description = "Pump",
        type = MeasureType.pump,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_test\input\measures\pump\pump.geojson"),
        discharge = us.UnitfulDischarge(value=1, units=us.UnitTypesDischarge.cfs))

# Create a measure object for Green infrastructure
attrs_greening = GreenInfrastructure(
        name= "green_infrastructure_storage",
        description = "Storage through green infrastructure",
        type = MeasureType.greening,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_test\input\measures\water_square\water_square.geojson"),
        volume = us.UnitfulVolume(value = 43975190.31512848, units = us.UnitTypesVolume.cf),
        height = us.UnitfulHeight(value = 3, units = us.UnitTypesLength.feet),
        percent_area = 100.0)

attrs_total_storage = GreenInfrastructure(
        name=  "total_storage_aggregation_area",
        description = "Total Storage through green infrastructure in aggr area",
        type = MeasureType.total_storage,
        selection_type = SelectionType.aggregation_area,
        aggregation_area_type = "aggr_lvl_2",
        aggregation_area_name = "name5",
        volume = us.UnitfulVolume(value = 100000000.0, units = us.UnitTypesVolume.cf),
)

attrs_water_square = GreenInfrastructure(
        name=  "water_square",
        description = "Water Square",
        type = MeasureType.water_square,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_test\input\measures\water_square\water_square.geojson"),
        volume = us.UnitfulVolume(value =  43975190.31512848, units = us.UnitTypesVolume.cf),
        height = us.UnitfulHeight(value = 3, units = us.UnitTypesLength.feet))  

## Create and save measures
# Create the neasures
fa.create_measure(attrs_floodwall.model_dump(exclude_none=True), type=attrs_floodwall.type)
fa.create_measure(attrs_pump.model_dump(exclude_none=True), type=attrs_pump.type)
fa.create_measure(attrs_greening.model_dump(exclude_none=True), type=attrs_greening.type)
fa.create_measure(attrs_total_storage.model_dump(exclude_none=True), type=attrs_total_storage.type)
fa.create_measure(attrs_water_square.model_dump(exclude_none=True), type=attrs_water_square.type)

# Save the measures to the database
fa.save_measure(attrs_floodwall)
fa.save_measure(attrs_pump)
fa.save_measure(attrs_greening)
fa.save_measure(attrs_total_storage)
fa.save_measure(attrs_water_square)

## **Step 3:** Create Impact Measures
To create a measure in Floodadapt we need to create a **Measure Object** with the FloodAdapt classes. Each object has the following parameters:

**Basic inputs**   
Basic inputs for <u>all</u> types of impact measures are:
- `name`: The name of the measure
- `description` (optionl): A description of the measure
- `type`: The type of measure
- `selection type`: The type of spatial shape of the measure
- `polygon file`: The file path to the spatial file of the measure
- `property_type`: The building type, e.g. residential, commercial, depends on you Delft-FIAT model classification*
- `aggregation_area_type` (optional): The name of the aggregation area category.
- `aggregation_area_name` (optional): The name of the specific aggregation area within the category.

****property_type***: You can apply the measure to a specific building occupancy. If e.g. you only want to buyout residential homes you can parse the building type of the residential buildings here. Make sure you parse the same string-value as you use in your Delft-FIAT model to describe that type of building. In this example notebook that would be "residential".

**Measure specific inputs**  
For each measure there are additional inputs that only apply to that measure in particular.   

1. **Elevate** specific inputs are:
    - `elevation`: The elevation value and unit (m,ft) of the assets and the vertical reference type (floodmap, datum).
2. **Buy out** specific inputs are:
    - *None*
3. **Floodproof** specific inputs are:
    - `elevation`: The elevation value and unit (m,ft) of the measure

Below we will create an `Measure`-object for each **impact measure**.

In [ ]:
# Create a measure object for Elevating Buildings
attrs_elevate = Elevate(
        name= "elevate_homes_2ft",
        description = "Elevate all residential buildings in aggregation area 1 by 2ft.",
        type = MeasureType.elevate_properties,
        selection_type = SelectionType.polygon,
        property_type = "residential",
        polygon_file =str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\raise_property_polygon\raise_property_polygon.geojson"),
        elevation = us.UnitfulLengthRefValue(value=2, units=us.UnitTypesLength.feet, type = us.VerticalReference.floodmap))

# Create a measure object for Buying out Buildings
attrs_buyout = Buyout(
        name= "buyout_all_buildings",
        description = "Buyout all buildings in a specific area.",
        type = MeasureType.buyout_properties,
        selection_type = SelectionType.aggregation_area,
        aggregation_area_type = "aggr_lvl_2",
        aggregation_area_name = "name5",
        property_type = "ALL",
)

# Create a measure object for Floodproofing Buildings
attrs_flood_proof = FloodProof(
        name= "floodproof_all_com",
        description = "Floodproofing all commercial buildings.",
        type = MeasureType.floodproof_properties,
        selection_type = SelectionType.all,
        property_type = "commercial",
        elevation = us.UnitfulLength(value=2, units=us.UnitTypesLength.feet))

## Create and save measures
# Create measures
fa.create_measure(attrs_elevate.model_dump(exclude_none=True), type=attrs_elevate.type)
fa.create_measure(attrs_buyout.model_dump(exclude_none=True), type=attrs_buyout.type)
fa.create_measure(attrs_flood_proof.model_dump(exclude_none=True), type=attrs_flood_proof.type)

# Save the measures to the database
fa.save_measure(attrs_elevate)
fa.save_measure(attrs_buyout)#
fa.save_measure(attrs_flood_proof)

Using the `get_measures()` method of the `FloodAdapt` class, we can check that the measures have been saved to the database.

In [6]:
pd.DataFrame(fa.get_measures())

,name,path,last_modification_date,description,objects,geometry
0,buyout,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 16:39:50.105548,,name='buyout' description='' type=<MeasureType...,id name ...
1,buyout_all_buildings,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:36.818332,Buyout all buildings in a specific area.,name='buyout_all_buildings' description='Buyou...,id name ...
2,elevate_homes_2ft,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:36.800811,Elevate all residential buildings in aggregati...,name='elevate_homes_2ft' description='Elevate ...,id ...
3,green_infrastructure_storage,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:31.841672,Storage through green infrastructure,name='green_infrastructure_storage' descriptio...,ge...
4,Pump,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:31.829148,Pump,name='Pump' description='Pump' type=<MeasureTy...,ge...
5,Seawall_12ft,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:31.819101,12ft Seawall,name='Seawall_12ft' description='12ft Seawall'...,id ...
6,total_storage_aggregation_area,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:31.851228,Total Storage through green infrastructure in ...,name='total_storage_aggregation_area' descript...,id name ...
7,water_square,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:02:31.866750,Water Square,name='water_square' description='Water Square'...,ge...


# 5. Copying and Editing a Measure in the database

If we want to edit small parts of a measure, it is easier to copy an existing measure and edit the copy. This way we do not have to create a new measure from scratch.

A measure can be copied in the database by using the `copy_measure()` method of the `FloodAdapt` class. This method takes three arguments: the name of the measure to be copied and the name and description of the new measure. Let's copy the measure we just created, having in mind that we want to make a measure for flood-proofing residential buildings.

In [9]:
# Copy measure
fa.copy_measure(old_name="floodproof_all_com", new_name="floodproof_all_res", new_description="Floodproofing all residential buildings.")

We can see that now a new measure with name "floodproof_all_res" has been created in the database. However, the actual attributes of the measure are still the same as the original measure. 

In [10]:
floodproof_res = fa.get_measure("floodproof_all_res")
floodproof_res

FloodProof(name='floodproof_all_res', description='Floodproofing all residential buildings.', type=<MeasureType.floodproof_properties: 'floodproof_properties'>, selection_type=<SelectionType.all: 'all'>, aggregation_area_type=None, aggregation_area_name=None, polygon_file=None, property_type='commercial', elevation=UnitfulLength(value=2.0, units=UnitTypesLength.feet))

We can directly edit the relevant attributes of the measure object. In this case, we want to change the type to "residential".

In [12]:
floodproof_res.property_type = "residential"

Then using the `edit_measure()` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Measure` object. The `name` field of the measure object provided will be used to identify which measure is going to be updated in the database, with the given Measure object attributes.

In [13]:
fa.edit_measure(floodproof_res)

Now we can verify that the measure has been updated in the database. The property type is now "residential".

In [15]:
floodproof_res = fa.get_measure("floodproof_all_res")
floodproof_res.property_type

'residential'

## **Step 4:** Create a Strategy
**Strategies** are combinations **measures**. They allow us to run an test multiple measures in a single model run. 

When creating a strategy we need to create a **Strategy object** with the following inputs:
1. `name`: The name of the strategy
2. `description` (optional): The description of the strategy
3. `measures`: A list of all the measure object names, you would like to use in that strategy. 

In [16]:
# Create a strategy object
attrs_strategy = Strategy(
        name= "pump_greening_flood_proof",
        description = "Strategy with pump, greening and floodproofing",
        measures = [attrs_pump.name, attrs_greening.name, attrs_flood_proof.name],
        )

# Create the stategy
fa.create_strategy(attrs_strategy .model_dump(exclude_none=True))

# Save the stategy
fa.save_strategy(attrs_strategy )

Using the `get_strategies()` method of the `FloodAdapt` class, we can check that the strategies have been saved to the database.

In [19]:
pd.DataFrame(fa.get_strategies())

,name,path,last_modification_date,description,objects
0,no_measures,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 16:39:50.348489,,name='no_measures' description='' measures=[]
1,pump_greening_flood_proof,C:\Users\rautenba\repos\FloodAdapt\docs\_data\...,2025-05-14 17:06:39.858401,"Strategy with pump, greening and floodproofing",name='pump_greening_flood_proof' description='...


In [18]:
strategy = fa.get_strategy("pump_greening_flood_proof")
strategy

Strategy(name='pump_greening_flood_proof', description='Strategy with pump, greening and floodproofing', measures=['Pump', 'green_infrastructure_storage', 'floodproof_all_com'])

# 5. Copying and Editing a Strategy in the database

If we want to edit small parts of a strategy, it is easier to copy an existing strategy and edit the copy. This way we do not have to create a new strategy from scratch.

A strategy can be copied in the database by using the `copy_strategy()` method of the `FloodAdapt` class. This method takes three arguments: the name of the strategy to be copied and the name and description of the new strategy. Let's copy the strategy we just created, having in mind that we want to remove the greening from the strategy and add a floodwall instead.

In [ ]:
# Copy strategy
fa.copy_strategy(old_name="pump_greening_flood_proof", new_name="pump_floodwall_flood_proof", new_description="Strategy with pump, flodwall and floodproofing.")

We can see that now a new strategy with name "pump_floodwall_flood_proof" has been created in the database. However, the actual attributes of the strtaegy are still the same as the original srtategy. 

In [ ]:
strategy_2 = fa.get_strategy("pump_floodwall_flood_proof")
strategy_2

We can directly edit the relevant attributes of the measure object. In this case, we want to change the type to "residential".

In [ ]:
strategy_2.measures = strategy_2.measures.pop(attrs_greening.name)
strategy_2.measures = strategy_2.measures.append(attrs_floodwall.name)

Then using the `edit_strategy()` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Strategy` object. The `name` field of the Strategy object provided will be used to identify which strategy is going to be updated in the database, with the given Strategy object attributes.

In [ ]:
fa.edit_strategy("pump_floodwall_flood_proof")

Now we can verify that the strategy has been updated in the database.

In [ ]:
strategy_2 = fa.get_strategy("pump_floodwall_flood_proof")
strategy_2.measures

### **DONE!** 
**Congratulations** you created all **measures** possible in FloodAdap and combined some of them into a **strategy**!